In [10]:
import pandas as pd
import requests
import datetime as dt
from dateutil import parser
import pytz
import json

In [11]:
OPENFX_URL = "https://marginalttdemowebapi.fxopen.net:8443/api/v2"

API_ID = "843d68ff-61fe-4bce-a347-9c17275e3665"
API_KEY = "Abg8D9wQnNh4GwjA"
API_SECRET = "ZYHSBDJZ45fRe6qFBATfRxMerPFMHBN564MfjpQPq4dqfAtz9jaTxnHBjCBX88pA"

SECURE_HEADER = {
    "Authorization": f"Basic {API_ID}:{API_KEY}:{API_SECRET}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

In [12]:
session = requests.Session()
session.headers.update(SECURE_HEADER)

In [13]:
full_url = lambda x: f"{OPENFX_URL}/{x}"

## Account Details

In [14]:
resp = session.get(full_url('account'))

In [15]:
resp.status_code

200

In [16]:
#print(json.dumps(resp.json(), indent=2))

## Symbols (Instruments)

In [17]:
# first get all of the symbols
resp = session.get(full_url('symbol'))
symbol_data = resp.json()

# print first 2, note the StatusGroupId=="Forex"
[print(json.dumps(x, indent=2)) for x in symbol_data[:2]]

# also we only want symbols where we can also load history data. For that there is the quotehistory/symbols
resp = session.get(full_url('quotehistory/symbols'))
his_symbol_data = resp.json()

print(his_symbol_data[:5])

# you can probaby see, some of the instruments are appended with "_L"
# in the API code we will filter for symbols that are in the symbol_data and are in his_symbol_data and do not have this L and have StatusGroupId=="Forex"

{
  "DefaultSlippage": 0.01,
  "MinCommission": 0,
  "LimitsCommission": 0.0015,
  "Commission": 0.0018,
  "TradeAmountStep": 1000.0,
  "MaxTradeAmount": 100000000,
  "MinTradeAmount": 1000.0,
  "IsLongOnly": false,
  "IsCloseOnly": false,
  "SwapEnabled": true,
  "IsTradeAllowed": true,
  "TripleSwapDay": 3,
  "SwapSizeLong": -9.73,
  "SwapSizeShort": 0.24,
  "Color": -10496,
  "ProfitCurrencyPrecision": 2,
  "MarginCurrencyPrecision": 2,
  "Precision": 5,
  "HiddenLimitOrderMarginReduction": 1,
  "StopOrderMarginReduction": 1,
  "MarginFactor": 1,
  "MarginHedged": 0.5,
  "ContractSize": 100000,
  "MarginMode": "Forex",
  "ProfitMode": "Forex",
  "SwapType": "Points",
  "CommissionType": "Percentage",
  "CommissionChargeType": "PerLot",
  "SlippageType": "Percent",
  "TradingMode": "Full",
  "ExtendedName": "EURUSD",
  "SecurityDescription": "Major Forex symbols",
  "SecurityName": "ECN FX Group1",
  "StatusGroupId": "Forex",
  "MinCommissionCurrency": "USD",
  "Schedule": "Forex",
 

## Perdiodicities (Granularities)

In [18]:
# for a given instrument (symbol) we can get the available candle granularities
resp = session.get(full_url('quotehistory/EURUSD/periodicities'))
print(resp.json())

['D1', 'H1', 'H4', 'M1', 'M15', 'M30', 'M5', 'MN1', 'S1', 'S10', 'W1']


## Candles

Candles are a little bit different than Oanda. <br><br>
The Good:
- Prices are floats!!
- Last available candle is in the response<br><br>

The bad:
- We have to make separate calls for ask and bid prices
- We have to specify a from date no matter what. It has to be a timestamp in ms format without timezone.
    - If the count we specify is negative it counts back from the date
    - If the count we specify is postive it counts forward from the date
- There is a 1000 candle limit per request

In [19]:
test_date = dt.datetime.utcnow()
past_date = parser.parse("2023-03-02T03:11:00")
print("test_date", test_date)
print("past_date", past_date)

test_date_ts = pd.Timestamp(test_date).timestamp()
past_date_ts = pd.Timestamp(past_date).timestamp()
print("test_date_ts", test_date_ts)
print("int(test_date_ts*1000)", int(test_date_ts*1000))
print("int(past_date_ts*1000)", int(past_date_ts*1000))

test_date 2024-06-04 10:26:49.157198
past_date 2023-03-02 03:11:00
test_date_ts 1717496809.157198
int(test_date_ts*1000) 1717496809157
int(past_date_ts*1000) 1677726660000


In [20]:
ts_conv = 1677726660000
pd.to_datetime(ts_conv, unit='ms')

Timestamp('2023-03-02 03:11:00')

In [21]:
LABEL_MAP = {
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Close': 'c',
}

# normal params
count = -10
granularity = "M15"
pair = "EURUSD"

# how far do we need to go back to get our candles
params = dict(
    timestamp=int(pd.Timestamp(dt.datetime.utcnow()).timestamp() * 1000),
    count=count
)

url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
bid_data = session.get(url, params=params).json()

url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
ask_data = session.get(url, params=params).json()


In [22]:
# bid_data and ask_data will contain (hopefully) a key called "Bars", with the candle data:
bid_data["Bars"][:2] # the first two

[{'Volume': 3051,
  'Close': 1.08878,
  'Low': 1.08846,
  'High': 1.08904,
  'Open': 1.08902,
  'Timestamp': 1717488000000},
 {'Volume': 2485,
  'Close': 1.08786,
  'Low': 1.08783,
  'High': 1.08907,
  'Open': 1.08879,
  'Timestamp': 1717488900000}]

In [23]:
# Now to convert them to a dataframe
# main points: Timestamp to a datetime.
# we'll need to make a DataFrame for ask, for bid, merge and calculate the mid

In [24]:
# a little utility to take in a candle and return it as and object
# for example, if we are working with bid prices
# price_label='bid'
# item= {'Volume': 1476, 'Close': 1.06064,  'Low': 1.06054,  'High': 1.06104,  'Open': 1.06081,  'Timestamp': 1677535200000}
# the returned object is: { 'time': datetime, 'bid_c': 1.06064,  'bid_l': 1.06054,  'bid_h': 1.06104,  'bid_o': 1.06081 }
def get_price_dict(price_label: str, item):
        data = dict(time=pd.to_datetime(item['Timestamp'], unit='ms'))
        for ohlc in LABEL_MAP.keys():
            data[f"{price_label}_{LABEL_MAP[ohlc]}"]=item[ohlc]
        return data

In [25]:
# let's make the lists of objects
AvailableTo = pd.to_datetime(bid_data['AvailableTo'], unit='ms')

bids = [get_price_dict('bid', item) for item in bid_data["Bars"]]
asks = [get_price_dict('ask', item) for item in ask_data["Bars"]]

In [26]:
# last 2
bids[-2:]

[{'time': Timestamp('2024-06-04 10:00:00'),
  'bid_o': 1.08717,
  'bid_h': 1.08717,
  'bid_l': 1.08654,
  'bid_c': 1.08666},
 {'time': Timestamp('2024-06-04 10:15:00'),
  'bid_o': 1.08665,
  'bid_h': 1.08694,
  'bid_l': 1.0865,
  'bid_c': 1.08664}]

In [27]:
# now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
df_bid = pd.DataFrame.from_dict(bids)
df_ask = pd.DataFrame.from_dict(asks)
df_merged = pd.merge(left=df_bid, right=df_ask, on='time')    

In [28]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-04 08:00:00,1.08902,1.08904,1.08846,1.08878,1.08903,1.08904,1.08846,1.08878
1,2024-06-04 08:15:00,1.08879,1.08907,1.08783,1.08786,1.08879,1.08908,1.08784,1.08786
2,2024-06-04 08:30:00,1.08787,1.08814,1.08740,1.08778,1.08787,1.08814,1.08740,1.08778
3,2024-06-04 08:45:00,1.08776,1.08815,1.08768,1.08809,1.08776,1.08816,1.08768,1.08809
4,2024-06-04 09:00:00,1.08809,1.08821,1.08693,1.08714,1.08809,1.08821,1.08693,1.08714
5,2024-06-04 09:15:00,1.08713,1.08771,1.08711,1.08744,1.08713,1.08771,1.08711,1.08744
6,2024-06-04 09:30:00,1.08744,1.08801,1.08744,1.08784,1.08744,1.08802,1.08744,1.08784
7,2024-06-04 09:45:00,1.08784,1.08790,1.08714,1.08718,1.08785,1.08790,1.08714,1.08718
8,2024-06-04 10:00:00,1.08717,1.08717,1.08654,1.08666,1.08717,1.08717,1.08654,1.08667
9,2024-06-04 10:15:00,1.08665,1.08694,1.08650,1.08664,1.08667,1.08694,1.08650,1.08664


In [29]:
# FINALLY calcuate the mid, and we are done
for i in ['_o', '_h', '_l', '_c']:
    df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']

In [30]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2024-06-04 08:00:00,1.08902,1.08904,1.08846,1.08878,1.08903,1.08904,1.08846,1.08878,1.089025,1.089040,1.088460,1.088780
1,2024-06-04 08:15:00,1.08879,1.08907,1.08783,1.08786,1.08879,1.08908,1.08784,1.08786,1.088790,1.089075,1.087835,1.087860
2,2024-06-04 08:30:00,1.08787,1.08814,1.08740,1.08778,1.08787,1.08814,1.08740,1.08778,1.087870,1.088140,1.087400,1.087780
3,2024-06-04 08:45:00,1.08776,1.08815,1.08768,1.08809,1.08776,1.08816,1.08768,1.08809,1.087760,1.088155,1.087680,1.088090
4,2024-06-04 09:00:00,1.08809,1.08821,1.08693,1.08714,1.08809,1.08821,1.08693,1.08714,1.088090,1.088210,1.086930,1.087140
5,2024-06-04 09:15:00,1.08713,1.08771,1.08711,1.08744,1.08713,1.08771,1.08711,1.08744,1.087130,1.087710,1.087110,1.087440
6,2024-06-04 09:30:00,1.08744,1.08801,1.08744,1.08784,1.08744,1.08802,1.08744,1.08784,1.087440,1.088015,1.087440,1.087840
7,2024-06-04 09:45:00,1.08784,1.08790,1.08714,1.08718,1.08785,1.08790,1.08714,1.08718,1.087845,1.087900,1.087140,1.087180
8,2024-06-04 10:00:00,1.08717,1.08717,1.08654,1.08666,1.08717,1.08717,1.08654,1.08667,1.087170,1.087170,1.086540,1.086665
9,2024-06-04 10:15:00,1.08665,1.08694,1.08650,1.08664,1.08667,1.08694,1.08650,1.08664,1.086660,1.086940,1.086500,1.086640


In [31]:
if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
    df_merged = df_merged[:-1]

In [32]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2024-06-04 08:00:00,1.08902,1.08904,1.08846,1.08878,1.08903,1.08904,1.08846,1.08878,1.089025,1.089040,1.088460,1.088780
1,2024-06-04 08:15:00,1.08879,1.08907,1.08783,1.08786,1.08879,1.08908,1.08784,1.08786,1.088790,1.089075,1.087835,1.087860
2,2024-06-04 08:30:00,1.08787,1.08814,1.08740,1.08778,1.08787,1.08814,1.08740,1.08778,1.087870,1.088140,1.087400,1.087780
3,2024-06-04 08:45:00,1.08776,1.08815,1.08768,1.08809,1.08776,1.08816,1.08768,1.08809,1.087760,1.088155,1.087680,1.088090
4,2024-06-04 09:00:00,1.08809,1.08821,1.08693,1.08714,1.08809,1.08821,1.08693,1.08714,1.088090,1.088210,1.086930,1.087140
5,2024-06-04 09:15:00,1.08713,1.08771,1.08711,1.08744,1.08713,1.08771,1.08711,1.08744,1.087130,1.087710,1.087110,1.087440
6,2024-06-04 09:30:00,1.08744,1.08801,1.08744,1.08784,1.08744,1.08802,1.08744,1.08784,1.087440,1.088015,1.087440,1.087840
7,2024-06-04 09:45:00,1.08784,1.08790,1.08714,1.08718,1.08785,1.08790,1.08714,1.08718,1.087845,1.087900,1.087140,1.087180
8,2024-06-04 10:00:00,1.08717,1.08717,1.08654,1.08666,1.08717,1.08717,1.08654,1.08667,1.087170,1.087170,1.086540,1.086665


In [33]:
# and breathe...

## Latest Prices

In [34]:
# here the endpoint needs space separated instruments rather than comma. Yes, in a URL that is a bit weird.
instruments_list = ["GBPJPY", "EURUSD", "EURNOK"]
url = full_url(f"tick/{' '.join(instruments_list)}")
print(url)

https://marginalttdemowebapi.fxopen.net:8443/api/v2/tick/GBPJPY EURUSD EURNOK


In [35]:
prices = session.get(url)
price_data = prices.json()

# you can see below that there are some differences to the Oanda Api in what comes back, imho this is a lot better
price_data

[{'Timestamp': 1717496812391,
  'IndicativeTick': False,
  'BestBid': {'Volume': 1875000000, 'Price': 1.08666, 'Type': 'Bid'},
  'BestAsk': {'Volume': 3000000000, 'Price': 1.08666, 'Type': 'Ask'},
  'TickType': 'Normal',
  'Symbol': 'EURUSD'},
 {'Timestamp': 1717496812391,
  'IndicativeTick': False,
  'BestBid': {'Volume': 300000000, 'Price': 197.656, 'Type': 'Bid'},
  'BestAsk': {'Volume': 800000000, 'Price': 197.668, 'Type': 'Ask'},
  'TickType': 'Normal',
  'Symbol': 'GBPJPY'},
 {'Timestamp': 1717496812391,
  'IndicativeTick': False,
  'BestBid': {'Volume': 50000000, 'Price': 11.49339, 'Type': 'Bid'},
  'BestAsk': {'Volume': 50000000, 'Price': 11.49467, 'Type': 'Ask'},
  'TickType': 'Normal',
  'Symbol': 'EURNOK'}]